**ОПРЕДЕЛЕНИЕ ПОТРЕБИТЕЛЬСКОЙ ЛОЯЛЬНОСТИ СРЕДИ КЛИЕНТОВ ДЛЯ ТЕЛЕКОММУНИКАЦИОННОЙ КОМПАНИИ.**

✅ **Описание проекта:**

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.  

Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
* 9-10 баллов — «cторонники» (англ. promoters);
* 7-8 баллов — «нейтралы» (англ. passives);
* 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: `% «сторонников» - % «критиков»`.
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). 
    
✅ **Цель исследования:**
    
1. Определить текущий уровень потребительской лояльности, или NPS среди клиентов из России.
2. Оформить результаты в визуальном представлении.
    
✅ **Описание данных:**

**Таблица `user`**

Содержит основную информацию о клиентах.

* `user_id`- Идентификатор клиента, первичный ключ таблицы
* `lt_day` - Количество дней «жизни» клиента
* `age` - Возраст клиента в годах
* `gender_segment` - Пол клиента (1 – женщина, 0 – мужчина)
* `os_name` - Тип операционной системы
* `cpe_type_name` - Тип устройства
* `location_id` - Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
* `age_gr_id` - Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
* `tr_gr_id` - Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
* `lt_gr_id` - Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
* `nps_score` - Оценка клиента в NPS-опросе (от 1 до 10)


**Таблица `location`**
Справочник территорий, в которых телеком-компания оказывает услуги.

* `location_id` - Идентификатор записи, первичный ключ
* `country` - Страна
* `city` - Город

**Таблица `age_segment`**
Данные о возрастных сегментах клиентов.

* `age_gr_id` - Идентификатор сегмента, первичный ключ
* `bucket_min` - Минимальная граница сегмента
* `bucket_max` - Максимальная граница сегмента
* `title` - Название сегмента

**Таблица `traffic_segment`**
Данные о выделяемых сегментах по объёму потребляемого трафика.

* `tr_gr_id` - Идентификатор сегмента, первичный ключ
* `bucket_min` - Минимальная граница сегмента
* `bucket_max` - Максимальная граница сегмента
* `title` - Название сегмента

**Таблица `lifetime_segment`**
Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.

* `lt_gr_id` - Идентификатор сегмента, первичный ключ
* `bucket_min` - Минимальная граница сегмента
* `bucket_max` - Максимальная граница сегмента
* `title` - Название сегмента


✅ **План работы:**
    
1. Подключение к базе;
2. Выгрузка данных;
3. Создание дашборда в Tableau для ответа на поставленные вопросы: 
* Как распределены участники опроса?
* Каких пользователей больше по возрасту?
* Каких пользователей больше по полу и возрасту?
* Каких пользователей больше: новых или старых?
* Пользователи из каких городов активнее участвовали в опросе?
* Какие группы пользователей наиболее и наименее лояльны к сервису?
* Какой общий NPS среди всех опрошенных?
* Как можно описать клиентов, которые относятся к группе cторонников?


## Импортируем библиотеки

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text

## Подключение к базе

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Выгрузка данных

In [3]:
query = """
WITH data_cte AS (
    SELECT
        u.user_id,
        u.lt_day,
        u.age,
        u.gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        SUBSTR(a.title, 3) AS age_segment,
        SUBSTR(t.title, 3) AS traffic_segment,
        SUBSTR(lt.title, 3) AS lifetime_segment,
        u.nps_score,
        CASE WHEN u.nps_score >= 9 THEN 'сторонники'
             WHEN u.nps_score >= 7 THEN 'нейтралы'
             ELSE 'критики'
        END AS nps_group,
        CASE WHEN u.lt_day <= 365 THEN 'True' ELSE 'False' END AS is_new
    FROM
        user AS u
        LEFT JOIN location AS l ON l.location_id = u.location_id
        LEFT JOIN age_segment AS a ON a.age_gr_id = u.age_gr_id
        LEFT JOIN traffic_segment AS t ON t.tr_gr_id = u.tr_gr_id
        LEFT JOIN lifetime_segment AS lt ON lt.lt_gr_id = u.lt_gr_id
)
SELECT *
FROM data_cte;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group,is_new
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники,False
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники,False
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники,False


In [5]:
df.to_csv('telecomm_csi_tableau_veria.csv', index=False)

## Создание дашборда в Tableau

Результаты исследования представленны по ссылке:

https://public.tableau.com/shared/3TTM6Z38Y?:display_count=n&:origin=viz_share_link